# Kartemap Application

Ziyu Tang, Xinlan Wu

In [14]:
import pandas as pd
from pandas import DataFrame
import numpy as np

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go

from graph.shortest_path import Dijkstra
from graph.network import Network

import warnings
warnings.filterwarnings('ignore')

In [15]:
pd.set_option('display.max_columns', None)

## Prepare data

#### Read data

In [16]:
df = pd.read_csv('data/final flight data with population.csv')
city_info = pd.read_csv('data/city_info.csv')

In [17]:
city_info.head()

,City,Population,Links,photo_link
0,Aberdeen,unknown,https://www.britannica.com/place/Aberdeen-Mary...,https://cdn.britannica.com/78/117478-050-6A0BF...
1,Abilene,"172,060",https://www.britannica.com/place/Abilene-Kansas,https://cdn.britannica.com/60/186160-050-0C856...
2,Akron,"703,479",https://www.britannica.com/place/Akron-Ohio,https://cdn.britannica.com/91/116691-004-37CFF...
3,Alamosa,unknown,https://www.britannica.com/place/Alamosa,https://cdn.britannica.com/30/172530-050-7C83F...
4,Albany,"880,381",https://www.britannica.com/place/Albany-Georgia,https://cdn.britannica.com/47/121747-050-5959B...


In [18]:
df = df.sort_values(['Source_City','Destination_City'])
df['Source_Popution'] = df['Source_Popution'].replace(np.nan, 'unknown', regex=True)
df['Destination_Popution'] = df['Destination_Popution'].replace(np.nan, 'unknown', regex=True)

df.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Source_Airport,Source_City,Source_Latitude,Source_Longitude,Source_Altitude,Destination_Airport,Destination_City,Destination_Latitude,Destination_Longitude,Destination_Altitude,Source_geometry,Destination_geometry,distance,Source_Popution,Destination_Popution
3612,DL,2009,ABR,5714,MSP,3858,Y,0,CRJ,Aberdeen Regional Airport,Aberdeen,45.449100,-98.421799,1302,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,44.882000,-93.221802,841,"(45.44910049438477, -98.42179870605469)","(44.882, -93.221802)",413.573939,unknown,"3,654,908"
518,AA,24,ABI,3718,DFW,3670,Y,0,ERD ER4 CRJ,Abilene Regional Airport,Abilene,32.411301,-99.681900,1791,Dallas Fort Worth International Airport,Dallas-Fort Worth,32.896801,-97.038002,607,"(32.4113006592, -99.68190002440001)","(32.896801, -97.038002)",253.808916,"172,060",unknown
8946,US,5265,ABI,3718,DFW,3670,NaN,0,ERD ER4,Abilene Regional Airport,Abilene,32.411301,-99.681900,1791,Dallas Fort Worth International Airport,Dallas-Fort Worth,32.896801,-97.038002,607,"(32.4113006592, -99.68190002440001)","(32.896801, -97.038002)",253.808916,"172,060",unknown
2576,AS,439,ADK,5959,ANC,3774,NaN,0,734 73Q,Adak Airport,Adak Island,51.877998,-176.645996,18,Ted Stevens Anchorage International Airport,Anchorage,61.174400,-149.996002,152,"(51.87799835205078, -176.64599609375)","(61.174400329589844, -149.99600219726562)",1918.725845,unknown,"396,317"
298,8D,1109,AKK,7160,ADQ,3531,NaN,0,CNA,Akhiok Airport,Akhiok,56.938702,-154.182999,44,Kodiak Airport,Kodiak,57.750000,-152.494003,78,"(56.9387016296, -154.182998657)","(57.75, -152.4940033)",136.025819,unknown,unknown


In [19]:
df_airport_1 = df[['Source_Airport','Source_City','Source_Latitude','Source_Longitude']]
df_airport_1 = df_airport_1.rename(columns={'Source_Airport':'Airport','Source_City':'City','Source_Latitude':'Latitude','Source_Longitude':'Longitude'})
df_airport_2 = df[['Destination_Airport','Destination_City','Destination_Latitude','Destination_Longitude']]
df_airport_2 = df_airport_2.rename(columns={'Destination_Airport':'Airport','Destination_City':'City','Destination_Latitude':'Latitude','Destination_Longitude':'Longitude'})
df_airport = pd.concat([df_airport_1,df_airport_2])
df_airport = df_airport.drop_duplicates(subset=['Airport','City'])
df_airport.head()

,Airport,City,Latitude,Longitude
3612,Aberdeen Regional Airport,Aberdeen,45.449100,-98.421799
518,Abilene Regional Airport,Abilene,32.411301,-99.681900
2576,Adak Airport,Adak Island,51.877998,-176.645996
298,Akhiok Airport,Akhiok,56.938702,-154.182999
2345,Akron Canton Regional Airport,Akron,40.916100,-81.442200


In [20]:
df_city = city_info.copy()

#### Create a dictionary of airport network

In [21]:
source_list = df['Source_Airport'].unique()
network_dict = {}
for i in source_list:
    new_df = df[df['Source_Airport'] == i][['Destination_Airport','distance']].set_index('Destination_Airport')
    new_df = new_df.rename(columns={'distance':i})
    dict_ = new_df.to_dict()
    network_dict.update(dict_)

#### Create relation options between departure city and departure airport

In [22]:
departure_list = df['Source_City'].unique()
departure_airport_dict = {}
for i in departure_list:
    new_ = df[df['Source_City'] == i]
    departure_airport_dict[i] = new_['Source_Airport'].unique().tolist()

#### Create relation options between arrival city and arrival airport

In [23]:
arrival_list = df['Destination_City'].unique()
arrival_airport_dict = {}
for i in arrival_list:
    new_ = df[df['Destination_City'] == i]
    arrival_airport_dict[i] = new_['Destination_Airport'].unique().tolist()

#### Create a function to read network data

In [24]:
def read_network_from_file(file_name, delimeter=','):
    """ Read from a file and build a network
    file_name: file to read from
    delimeter: delimeter that separates fields
    """
    cities = list()
    distances = dict()

    f = open(file_name, 'r')
    lines = f.readlines()
    for line in lines:
        fields = line.rstrip().split(delimeter)
        city_1 = fields[0].strip(' ')
        city_2 = fields[1].strip(' ')
        distance = float(fields[2])

        # build the list of cities
        if city_1 not in cities:
            cities.append(city_1)
        if city_2 not in cities:
            cities.append(city_2)

        # build the dictionary based on city distances
        if cities.index(city_1) not in distances.keys():
            distances[cities.index(city_1)] = {cities.index(city_2): distance}
        if cities.index(city_2) not in distances[cities.index(city_1)].keys():
            distances[cities.index(city_1)][cities.index(city_2)] = distance

    return cities, distances

file_name = 'data/flight network.csv'
cities, distances = read_network_from_file(file_name)

## App

In [25]:
[{"label":i,"value":i}for i in df.Source_City.unique()]

[{'label': 'Aberdeen', 'value': 'Aberdeen'},
 {'label': 'Abilene', 'value': 'Abilene'},
 {'label': 'Adak Island', 'value': 'Adak Island'},
 {'label': 'Akhiok', 'value': 'Akhiok'},
 {'label': 'Akron', 'value': 'Akron'},
 {'label': 'Akutan', 'value': 'Akutan'},
 {'label': 'Alakanuk', 'value': 'Alakanuk'},
 {'label': 'Alamosa', 'value': 'Alamosa'},
 {'label': 'Albany', 'value': 'Albany'},
 {'label': 'Albuquerque', 'value': 'Albuquerque'},
 {'label': 'Alexandria', 'value': 'Alexandria'},
 {'label': 'Allakaket', 'value': 'Allakaket'},
 {'label': 'Allentown', 'value': 'Allentown'},
 {'label': 'Alliance', 'value': 'Alliance'},
 {'label': 'Alpena', 'value': 'Alpena'},
 {'label': 'Altoona', 'value': 'Altoona'},
 {'label': 'Amarillo', 'value': 'Amarillo'},
 {'label': 'Ambler', 'value': 'Ambler'},
 {'label': 'Anaktuvuk Pass', 'value': 'Anaktuvuk Pass'},
 {'label': 'Anchorage', 'value': 'Anchorage'},
 {'label': 'Aniak', 'value': 'Aniak'},
 {'label': 'Anvik', 'value': 'Anvik'},
 {'label': 'Appleton

In [27]:
#============================================== Layout ==============================================#

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])

layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
    title="Map",
    mapbox=dict(
        accesstoken = 'pk.eyJ1Ijoieml5dXQiLCJhIjoiY2toNGFqamtwMDR6ZDM0cWg0MzJ3NWxyMiJ9.bH9W0iiVgZMF_s5VyeoZ4g',
        style="light",
        center=dict(lon=-95, lat=39.5),
        zoom=3.5,
    ),
)

# Component of input group

controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Start City"),
                dcc.Dropdown(
                    options=[{"label":i,"value":i}for i in df.Source_City.unique()],
                    value = 'Choose a start city',
                    id="departure_select",
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Destination City"),
                dcc.Dropdown(
                    #options=[{"label":i,"value":i} for i in df.Destination_City.unique()],
                    #value="Choose a destination city",
                    id="arrival_select",
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Start Airport"),
                dcc.Dropdown(
                    id="departure_airport_options"
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Destination Airport"),
                dcc.Dropdown(
                    id="arrival_airport_options"
                ),
            ]
        ),
        dbc.Button("Submit", outline=False, color="primary", className="mr-1",id='submit-button-state',n_clicks = 0),
    ],
    body=True,
)


# Component of source city info

first_card = dbc.Card(
        [html.Div(id='source-city-image'),
        dbc.CardBody(
        [
            html.H5(html.Div(id='display-source-city-name'),
                style={
                      'textAlign': 'left',
                      'color': '#173F5F',
                      'fontSize': 25
                }),
            html.P(html.Div(id='display-source-city-info')),
            html.Div(id='display-source-city-link')
            #dbc.Button("Go to see city info", color="primary"),
        ]
    )
      ]
)

# Component of destination city info

second_card = dbc.Card(
    [html.Div(id='arrival-city-image'),
    dbc.CardBody(
        [
            html.H5(html.Div(id='display-arrival-city-name'),
                style={
                      'textAlign': 'left',
                      'color': '#173F5F',
                      'fontSize': 25
                }),
            html.P(html.Div(id='display-arrival-city-info')),
            html.Div(id='display-destination-city-link')
            #dbc.Button("Go to see city info", color="primary"),
        ]
    )
    ]
)

# Layout of overall web application 

app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(html.H1("Kartemap - An Airport Network Analysis Application"))
        ),
        
        dbc.Row(
            dbc.Col(html.P("Make your air trip fast and easy!"),
                    style={
                      'textAlign': 'left',
                      'color': '#173F5F',
                      'fontSize': 25
                })
        ),
        
        dbc.Row(
            [
                dbc.Col(controls, md=3),
                dbc.Col(dcc.Graph(id="map"), md=9),
                dcc.Store(id='memory-output_route'),
                dcc.Store(id='memory-output_distance')
            ],
            align="center",
        ),
        
        html.Br(),
        
        dbc.Row(
            dbc.Col(
                dbc.Card(
                          dbc.CardBody(html.Div(id='display-selected-values'),
                style={
                      'textAlign': 'left',
                      'color': '#173F5F',
                      'fontSize': 18
                }),
                          color="secondary", inverse=True, className="mb-3",
        )
                
            )
        ),
        
        html.Br(),
        
        dbc.Row([dbc.Col(first_card, width=6), dbc.Col(second_card, width=6)])
    ],
    
    id="main-container",
    style={"display": "flex", "flex-direction": "column"},
    fluid=True
)
  
#============================================== Callbacks ==============================================#
    
# callback of airport according to the city input
@app.callback(
    Output('arrival_select', 'options'),
    [Input('departure_select', 'value')])
def set_arrival_select_options(selected_departure_city):
    arrival_city_list = df.Destination_City.unique().tolist()
    arrival_city_list.remove(selected_departure_city)
    return [{'label': i, 'value': i} for i in arrival_city_list]


@app.callback(
    Output('departure_airport_options', 'options'),
    [Input('departure_select', 'value')])
def set_departure_airport_options(selected_departure_city):
    return [{'label': i, 'value': i} for i in departure_airport_dict[selected_departure_city]]


@app.callback(
    Output('departure_airport_options', 'value'),
    [Input('departure_airport_options', 'options')])
def set_departure_airport_value(available_departure_options):
    return available_departure_options[0]['value']


@app.callback(
    Output('arrival_airport_options', 'options'),
    [Input('arrival_select', 'value')])
def set_arrival_airport_options(selected_arrival_city):
    return [{'label': i, 'value': i} for i in arrival_airport_dict[selected_arrival_city]]


@app.callback(
    Output('arrival_airport_options', 'value'),
    [Input('arrival_airport_options', 'options')])
def set_arrival_airport_value(available_arrival_options):
    return available_arrival_options[0]['value']
    

    
## callback of the shorest route and the distance
# route:
@app.callback(
    Output(component_id="memory-output_route",component_property="data"),
    [Input(component_id="departure_airport_options",component_property="value"),
     Input(component_id="arrival_airport_options", component_property="value"),
    ])

def shortest_path(initial, end):
    network = Network()
    network.add_nodes(cities)
    for connection in distances.items():
        frm = cities[connection[0]]
        for connection_to in connection[1].items():
            network.add_edge(frm, cities[connection_to[0]], connection_to[1])


    start_city_index = cities.index(initial)
    start_city = network.get_nodes()[start_city_index]


    Dijkstra.compute(network, network.get_node(start_city))

    target_city = network.get_node(end)
    path = [target_city.get_name()]
    Dijkstra.compute_shortest_path(target_city, path)
    route = path[::-1]
    return route

# distance
@app.callback(
    Output(component_id="memory-output_distance",component_property="data"),
    [Input(component_id="departure_airport_options",component_property="value"),
     Input(component_id="arrival_airport_options", component_property="value"),
    ])

def shortest_path(initial, end):
    network = Network()
    network.add_nodes(cities)
    for connection in distances.items():
        frm = cities[connection[0]]
        for connection_to in connection[1].items():
            network.add_edge(frm, cities[connection_to[0]], connection_to[1])


    start_city_index = cities.index(initial)
    start_city = network.get_nodes()[start_city_index]


    Dijkstra.compute(network, network.get_node(start_city))

    target_city = network.get_node(end)
    path = [target_city.get_name()]
    Dijkstra.compute_shortest_path(target_city, path)
    distance = round(target_city.get_weight(),2)
    return distance

# callback of map

default_map = go.Figure(go.Scattermapbox())
default_map.update_layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        mapbox = {
                  'accesstoken': 'pk.eyJ1Ijoieml5dXQiLCJhIjoiY2toNGFqamtwMDR6ZDM0cWg0MzJ3NWxyMiJ9.bH9W0iiVgZMF_s5VyeoZ4g',
                  'style': "outdoors", 
                  'center': {'lon': -95, 'lat': 39.5},
                  'zoom': 3.5},
        showlegend = False)


@app.callback(Output('map', 'figure'),
            [Input('submit-button-state','n_clicks')],
            [State('departure_airport_options', 'value'),
             State('arrival_airport_options', 'value'),
             State("memory-output_route","data")])

def update_figure(n_clicks,departure_select,arrival_select,path):            
    if n_clicks:
        lat_list = []
        lon_list = []

        for i in path:
            lat_list.append(list(df_airport.loc[df_airport.Airport==i].Latitude)[0])
            lon_list.append(list(df_airport.loc[df_airport.Airport==i].Longitude)[0])
        
        marker_list = ['circle'] * len(path)
        text_list = [''] * len(path)
        marker_list[0] = 'airport'
        marker_list[-1] = 'star'
    
        for i in range(0,len(path)):
            text_list[i] = list(df_airport.loc[df_airport.Airport==path[i]].City)[0]
        text_list[0] = 'Departure'
        text_list[-1] = 'Arrival'
    
        fig = go.Figure(go.Scattermapbox(
        mode =  "markers+text+lines",
        lat = lat_list,
        lon = lon_list,
        marker = {'size': 10, 'symbol': marker_list},
        text = text_list,textposition = "bottom right"))
    
        fig.update_layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        mapbox = {
                  'accesstoken': 'pk.eyJ1Ijoieml5dXQiLCJhIjoiY2toNGFqamtwMDR6ZDM0cWg0MzJ3NWxyMiJ9.bH9W0iiVgZMF_s5VyeoZ4g',
                  'style': "outdoors", 
                  'center': {'lon': -95, 'lat': 39.5},
                  'zoom': 3.5},
        showlegend = False)

        return fig
    else:
        return default_map

# callback of sentence

@app.callback(Output('display-selected-values', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('departure_select', 'value'),
             State('arrival_select', 'value'),
             State('memory-output_distance','data')])

def set_display_children(n_clicks,selected_start, selected_arrival,distance):
    if n_clicks:
        return u'The shortest airline route from {} to {} is {} KM.'.format(selected_start, selected_arrival,distance)
    

# callback of source/arrival city name, info, image, and link
@app.callback(Output('display-source-city-name', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('departure_select', 'value')])

def display_source_city_name(n_clicks,selected_start):
    if n_clicks:
        return selected_start

@app.callback(Output('display-source-city-info', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('departure_select', 'value')])

def display_source_city_info(n_clicks,selected_start):
    if n_clicks:
        source_population = list(df_city.loc[df_city.City==selected_start].Population)[0]
        return u'The population of {} is {}.'.format(selected_start, source_population)

    
    
@app.callback(Output('display-arrival-city-name', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('arrival_select', 'value')])

def display_source_city_name(n_clicks,selected_arrival):
    if n_clicks:
        return selected_arrival

@app.callback(Output('display-arrival-city-info', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('arrival_select', 'value')])

def display_source_city_info(n_clicks,selected_arrival):
    if n_clicks:
        arrival_population = list(df_city.loc[df_city.City==selected_arrival].Population)[0]
        return u'The population of {} is {}.'.format(selected_arrival, arrival_population)
    

    
    
@app.callback(Output('source-city-image', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('departure_select', 'value')])

def display_source_city_image(n_clicks,selected_start):
    if n_clicks:
        source_city_image = list(df_city.loc[df_city.City==selected_start].photo_link)[0]
        return dbc.CardImg(src=source_city_image, top=True)

    
@app.callback(Output('arrival-city-image', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('arrival_select', 'value')])

def display_source_city_image(n_clicks,selected_arrival):
    if n_clicks:
        arrival_city_image = list(df_city.loc[df_city.City==selected_arrival].photo_link)[0]
        return dbc.CardImg(src=arrival_city_image, top=True)



@app.callback(Output('display-source-city-link', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('departure_select', 'value')])

def display_source_city_link(n_clicks,selected_start):
    if n_clicks:
        source_city_link = list(df_city.loc[df_city.City==selected_start].Links)[0]
        return dbc.CardLink('Go to explore city info', href = source_city_link,target = 'blank')

    
@app.callback(Output('display-destination-city-link', 'children'),
            [Input('submit-button-state','n_clicks')],
            [State('arrival_select', 'value')])

def display_source_city_link(n_clicks,selected_arrival):
    if n_clicks:
        arrival_city_link = list(df_city.loc[df_city.City==selected_arrival].Links)[0]
        return dbc.CardLink('Go to explore city info', href = arrival_city_link,target = 'blank')
    
    
    

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False,dev_tools_ui=False,dev_tools_props_check=False,\
                   port=8050, host='127.0.0.1')  

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
